## Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

## Load model and allocate tensors

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

## Make Detections

In [27]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _,frame=cap.read()
    
    #Reshape Image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image = tf.cast(img,dtype=tf.float32)
    
    #setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    #Make predictions
    interpreter.set_tensor(input_details[0]['index'],np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    #print(keypoints_with_scores)
    
    #Rendering
    draw_connections(frame,keypoints_with_scores,EDGES,0.4)
    draw_keypoints(frame,keypoints_with_scores,0.4)
    cv2.imshow('MoveNet Lightning',cv2.flip(frame,1))
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

**How to get data**

In [13]:
right_eye = keypoints_with_scores[0][0][2]
np.array(right_eye[:-1]*[480,640]).astype(int)

array([209, 335])

In [15]:
shaped = np.squeeze(interpreter.get_tensor(output_details[0]['index'])*[480,640,1])
#print(shaped)
for kp in shaped:
    ky,kx,kp_conf = kp
    print(int(kx),int(ky),kp_conf)

379 236 0.7692314386367798
405 204 0.7363525629043579
335 209 0.47225403785705566
432 214 0.6654006242752075
289 228 0.7968537211418152
531 310 0.7863597869873047
215 331 0.8477886915206909
595 420 0.03343367576599121
190 422 0.03146308660507202
460 300 0.00731930136680603
292 383 0.0031796395778656006
259 208 0.009642869234085083
268 163 0.008072346448898315
618 315 0.01788806915283203
214 326 0.01835101842880249
651 245 0.009585678577423096
650 235 0.006071269512176514


## Draw Keypoints

In [16]:
def draw_keypoints(frame,keypoints,confidence):
    y,x,c = frame.shape
    shaped = np.squeeze(keypoints*[y,x,1])
    
    for kp in shaped:
        ky,kx,kp_conf = kp
        if kp_conf>confidence:
            cv2.circle(frame,(int(kx),int(ky)),4,(0,255,0),-1)
            

## Draw Edges

In [22]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
#All connections

In [25]:
def draw_connections(frame,keypoints,edges,confidence):
    y,x,c = frame.shape
    shaped = np.squeeze(keypoints*[y,x,1]) 
    for edge,color in edges.items():
        p1,p2 = edge
        y1,x1,c1 = shaped[p1]
        y2,x2,c2 = shaped[p2]
        
        if(c1>confidence and c2>confidence):
            cv2.line(frame,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),2)